In [ ]:
import json
import sys
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np

# Import modules
import matplotlib as mpl
import numpy as np
from scipy.stats import fisher_exact
from statsmodels.stats.contingency_tables import Table2x2
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

sys.path.append("../config/")
from config_plot import config_plot_params

# 4B

In [ ]:
# genes in discovery

unique_set = {'ABL2',
 'AFF3',
 'APC',
 'AR',
 'ARID2',
 'ASXL1',
 'ATE1',
 'ATM',
 'BCOR',
 'CALR',
 'CBL',
 'CDKN1B',
 'CEBPA',
 'CHEK2',
 'CSF3R',
 'CTCF',
 'CUX1',
 'DNM2',
 'DNMT3A',
 'DNMT3B',
 'ERCC2',
 'ERF',
 'ETNK1',
 'EZH2',
 'FLT3',
 'FOXP1',
 'GATA2',
 'GNAS',
 'GNB1',
 'IDH1',
 'IDH2',
 'IKZF1',
 'JAK2',
 'KDM5C',
 'KDM6A',
 'KIT',
 'KMT2C',
 'KMT2D',
 'KRAS',
 'LZTR1',
 'MDM4',
 'MGA',
 'MKL1',
 'MPL',
 'MYCN',
 'MYD88',
 'MYO5A',
 'NBPF1',
 'NF1',
 'NOTCH1',
 'NPM1',
 'NRAS',
 'PABPC1',
 'PCMTD1',
 'PHF6',
 'PPARG',
 'PPM1D',
 'PTPN11',
 'PTPRD',
 'PUF60',
 'RAD21',
 'RET',
 'RUNX1',
 'SDHAF2',
 'SF3B1',
 'SH2B3',
 'SMC1A',
 'SRSF2',
 'STAG2',
 'STAT3',
 'STAT5B',
 'SUZ12',
 'TET2',
 'TLR1',
 'TMEM127',
 'TP53',
 'TP63',
 'U2AF1',
 'ZRSR2'}

In [ ]:
from config import OHSU_DATA_MUTATIONS, OHSU_DATA_CLINICAL

# BeatAML mutations from cbioportal
muts_cbioport = pd.read_csv(OHSU_DATA_MUTATIONS, sep ='\t')
muts_cbioport['SAMPLE'] = muts_cbioport['Tumor_Sample_Barcode'].apply(lambda x : x.split('_')[-1])

# Metadata from cbioportal
df_metadata_beat = pd.read_csv(OHSU_DATA_CLINICAL, sep ='\t', skiprows=4)

C_A = ['15-00056', '14-00527', '16-00118']

df_metadata_beat['SAMPLE'] = df_metadata_beat['SAMPLE_ID'].apply(lambda x : x.split('_')[-1])

sample_taml_beat = df_metadata_beat[(df_metadata_beat['SPECIFIC_DIAGNOSIS_AT_INCLUSION']=='Therapy-related myeloid neoplasms')]['SAMPLE'].unique()

muts_cbio = muts_cbioport[['Hugo_Symbol', 'Chromosome', 'Start_Position', 'Consequence','Variant_Type', 'short_aa_change', 
           'Tumor_Sample_Barcode', 'Reference_Allele', 'Tumor_Seq_Allele2', 'SAMPLE']].drop_duplicates()

muts_cbio['tAML'] = muts_cbio['SAMPLE'].apply(lambda x : 'tAML' if x in sample_taml_beat else 'primaryAML')

In [ ]:
# somatic mutations found in blood in HMF and TCGA, from:
# Discovering the drivers of clonal hematopoiesis, https://www.biorxiv.org/content/10.1101/2020.10.22.350140v1

coding_variants_merged = pd.read_csv('/home/opich/bg/temp/merged_hmf_tcga.gz', sep ='\t')

In [ ]:
# dictionary with cytotoxic-related drugs in HMF. 
cytotox = json.load(open('/home/opich/bg/mutpro/src/reverse_calling/citotoxic.json'))
treated_full = coding_variants_merged[coding_variants_merged['NEW_S'].isin(cytotox)]
untreated_full = coding_variants_merged[~coding_variants_merged['NEW_S'].isin(cytotox)]

In [ ]:
set_muts_CH = treated_full[~treated_full['Consequence'].isin(['synonymous_variant', 'inframe_deletion', 
                                                            'inframe_insertion'])]

set_muts_CH["short_aa_change"] = set_muts_CH["Protein_position"] + set_muts_CH["Amino_acids"]

set_muts_taml = muts_cbio[muts_cbio['tAML']=='tAML']
set_muts_taml = set_muts_taml[~set_muts_taml["short_aa_change"].isnull()]
dic_compar = defaultdict(dict)

for gene in unique_set:
    
    muts_CH = len(set_muts_CH[(set_muts_CH['SYMBOL']==gene)]['SAMPLE'].unique())
    muts_tAML = len(set_muts_taml[set_muts_taml['Hugo_Symbol']==gene]['SAMPLE'].unique())
        
    if gene == "DNMT3A":
        
        muts_CH = len(set_muts_CH[(set_muts_CH['SYMBOL']==gene)&(set_muts_CH["short_aa_change"].str.contains('882R'))]['SAMPLE'].unique())
        muts_tAML = len(set_muts_taml[(set_muts_taml['Hugo_Symbol']==gene)&(set_muts_taml["short_aa_change"].str.contains('R882'))]['SAMPLE'].unique())
        dic_compar['CH']["DNMT3A p.R882"] = muts_CH
        dic_compar['tAML']["DNMT3A p.R882"] = muts_tAML
        muts_CH = len(set_muts_CH[(set_muts_CH['SYMBOL']==gene)&(~set_muts_CH["short_aa_change"].str.contains('882R'))]['SAMPLE'].unique())
        muts_tAML = len(set_muts_taml[(set_muts_taml['Hugo_Symbol']==gene)&(~set_muts_taml["short_aa_change"].str.contains('R882'))]['SAMPLE'].unique())
        dic_compar['CH']["DNMT3A other"] = muts_CH
        dic_compar['tAML']["DNMT3A other"] = muts_tAML

    elif (muts_CH>0)&(muts_tAML>0):
        dic_compar['CH'][gene] = muts_CH
        dic_compar['tAML'][gene] = muts_tAML

res = pd.DataFrame(dic_compar)

total_prim = res['CH'].sum()
total_tAML = res['tAML'].sum()
keep_res = {}

xlab = []
for ix, (i, row) in enumerate(res.sort_values(by='tAML', ascending = False).head(40).iterrows()):
    t = Table2x2([[row['tAML'], total_tAML],
                 [row['CH'], total_prim]])
    pval = t.oddsratio_pvalue()
    o = t.oddsratio_confint()
    cmin, cmax = o
    odds = t.oddsratio
    
    edge = 'white'
    color = 'orange'

    if odds >1:
        color = 'darkred'
    
    if pval <0.05:
        edge = 'black'
    
    keep_res[i] = (np.log2(odds), np.log2(cmin), np.log2(cmax), pval )


In [ ]:
tt_df = pd.DataFrame(keep_res, index = ['Val', 'LC', 'HC', 'pvalue']).T

qvals = multipletests(tt_df.pvalue, alpha=0.05, method='fdr_bh')
tt_df['QVAL'] = qvals[1]
#res_reg = age_gene.loc[wanted_genes[:10]].sort_values(by='Val', ascending = True)
res_reg = tt_df.sort_values(by='pvalue', ascending = True).head(20)

config_params(5)
fig, ax = plt.subplots(1, 1, figsize = (1.5,2.5))
labels = []
plt.title('t-CH vs t-AML')
for ix, (i, row) in enumerate(res_reg.sort_values(by='pvalue', ascending = False).iterrows()):
    
    edgecolor = 'white'
    if row['QVAL']<0.1:
        edgecolor = 'black'
    
    color = 'orange'
    if row['Val']>0:
        color = 'darkred'
    plt.scatter( row['Val'], ix, color = color, edgecolor = edgecolor)
    plt.plot([row['LC'], row['HC']], [ix, ix], color = color)
    labels.append(i)

plt.yticks(np.arange(len(res_reg)), ["$\it{0}$".format(lab) for lab in labels] )
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.vlines(0, 0, len(res_reg), ls = '--', color = 'grey')
#plt.xlim(0, 7)
plt.xlabel('Log2(odds-ratio)')
#plt.savefig('/home/opich/bg/temp/odds_t-CH_tAML.svg')
plt.show()

# 4C

In [ ]:
import sys
import itertools
import pickle, gzip
sys.path.append("../config/")
from config import SIG_ALL_AML_EXPOSURES, SIG_TREATMENT_DRUG_LVL

In [ ]:
AML_exposures = pd.read_csv(SIG_ALL_AML_EXPOSURES, sep ='\t', index_col = 0).T

In [ ]:
set_H = AML_exposures[(AML_exposures['1_0.97_SBS31-0.95']>500)&
                     (AML_exposures['2_0.97_NA']>0)]['2_0.97_NA'].tolist()
set_CIS = AML_exposures[(AML_exposures['1_0.97_SBS31-0.95']>500)&
                     (AML_exposures['2_0.97_NA']>0)]['1_0.97_SBS31-0.95'].tolist()
fig, ax = plt.subplots(1, 1, figsize = (1,1))

plt.scatter(np.zeros(len(set_H)), set_H, color = 'grey', label = 'HSC', s = 5)
plt.scatter(np.ones(len(set_CIS)), set_CIS, color = 'darkred', label = 'Platinum', s = 5)

for v1, v2 in zip(set_H, set_CIS):
    
    plt.plot([0, 1], [v1, v2], color = 'grey', lw = 0.2, ls = '--')

plt.ylabel('Number of mutations')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.legend(bbox_to_anchor = [1,1])
#plt.savefig('/home/opich/bg/temp/scatter_sigs.svg')
plt.xlim(-0.5, 1.5)
plt.show()

In [ ]:
# annotated variants from CH
coding_variants_merged = pd.read_csv('/home/opich/bg/temp/coding_variants_merged.tsv.gz', sep ='\t')

In [ ]:
# path to signature extraction of CH variants
CH_exposures = pd.read_csv('/home/opich/bg/reverse_calling/data/signatures/reverse_calling/SigProfiler/exposures/basic_filters_mosaicfull.snvs/basic_filters_mosaicfull.snvs.exposures.tsv', 
                           sep ='\t', index_col = 0).T

treatment_sample_level = pickle.load(gzip.open(SIG_TREATMENT_DRUG_LVL))
samples_t_cis = list(treatment_sample_level['PAN']['Cisplatin']) +
                list(treatment_sample_level['PAN']['Carboplatin'])

new_ix = [c.split('_')[0] for c in CH_exposures.index]
CH_exposures.index = new_ix

sexp = CH_exposures.loc[[s for s in samples_t_cis if s in CH_exposures.index]]

ppm1d_affected = set(coding_variants_merged[coding_variants_merged['GENE']=='PPM1D']['NEW_S'].tolist())

In [ ]:
v = [sexp[sexp['4_0.99_NA']>500]['4_0.99_NA'].tolist()]

config_params(5)
np.random.seed(1)
fig, ax = plt.subplots(1, 1, figsize = (0.8,1))
v = [sexp[sexp['4_0.99_NA']>0]['4_0.99_NA'].tolist()]
index_persons = sexp[sexp['4_0.99_NA']>0].index.tolist()

cols = ['grey' if i not in ppm1d_affected else 'orange' for i in index_persons]
size = [1 if i not in ppm1d_affected else 4 for i in index_persons]
x = list(itertools.chain.from_iterable([[0 + i] * len(l) for i, l in enumerate(v)]))
x += np.random.normal(0, 0.05, size=len(x))
y = list(itertools.chain.from_iterable(v))


ax=sns.boxplot(data=v, showfliers=False, color = 'white',
            medianprops={'color':'black'}, linewidth = 0.4, ax = ax) 
ax.scatter(x, y, s=1, alpha=0.5,
              color=cols,  )  # scatter zorder=5

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.ylabel('HSC mutations')
plt.ylim(0, 600)

plt.show()

# 4D

In [ ]:
import os
import tqdm
import gzip
import pickle
import glob
import sys

import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import itertools as itt

import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap = cm.get_cmap('Oranges')

# DON'T USED
#from utils import get_noisy_catalogue, Exposures, Processes, SignatureInjector, foreign_signatures

In [ ]:
sys.path.append("../config/")
from config_plot import config_plot_params

In [ ]:
from config import M_SIG_ACT_OBSERVED_UNTREATED, RESULT_CATALOGUE, BASIC_FILTERS

# Parse mSigAct results (true cases)

In [ ]:
def formatting(sig):
    
    """
    Some formatting rules for more clear notation
    """
    
    if 'X' in sig:
        sig = sig[1:]
    if 'SBS17b' in sig:
        sig = 'SBS17b'
    
    return sig

In [ ]:
df_list = []

for fn in glob.glob(os.path.join(M_SIG_ACT_OBSERVED_UNTREATED, RESULT_CATALOGUE)):
    
    if 'null' in fn:
        injected = 0
        foreign = formatting('.'.join(os.path.basename(fn).split('mSigAct')[-1].split('.')[1:-1]))
        
    else:
        injected = os.path.basename(fn).split('.tsv')[0].split('_')[-1]
        foreign  = formatting('_'.join(os.path.basename(fn).split('.tsv')[0].split('_')[1:-1]))
    
    df = pd.read_csv(fn, sep='\t')
    foreign_col_label = df.columns.tolist()[-1]
    df.rename(columns={foreign_col_label: formatting(foreign_col_label)}, inplace=True)
    
    df['injected'] = injected
    df['foreign'] = foreign
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'sample'}, inplace=True)
    
    df_list.append(df[['pval', 'sample', 'injected', 'foreign', formatting(foreign_col_label)]])

results = pd.concat(df_list)
results.fillna(value=0, inplace=True)
results['injected'] = results['injected'].apply(int)

# Reconstructed exposure by injected level

In [ ]:
reconstructed = {}
for sig in results['foreign'].unique():
    for inj in results['injected'].unique():
        df = results[(results['foreign'] == sig) & (results['injected'] == inj)]
        reconstructed[(sig, inj)] = df.loc[:, [col for col in df.columns if col not in ['pval', 'sample', 'injected', 'foreign']]].sum(axis=1).tolist()

In [ ]:
def run_plot(save=False):

    fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(20, 3))

    foreign = ['SBS17b', 'SBS35', 'oxaliplatin', 'SBS31']
    exposures = [2, 5, 7, 10, 15, 20, 25]
    colors = list(map(cmap, [(e+3)/25 for e in exposures]))

    for i, sig in enumerate(foreign):

        v0 = np.array(reconstructed[(sig, 0)])
        g0 = sns.kdeplot(v0, ax=ax[i], bw_adjust=0.4, color='green', 
                         shade=False, cumulative=True, label='zero injected', lw=5)

        for j, exp in enumerate(exposures):

            v1 = np.array(reconstructed[(sig, exp)])
            ks, pval = scipy.stats.ks_2samp(v0, v1)
            g = sns.kdeplot(v1, ax=ax[i], bw_adjust=0.4, color=colors[j],
                            shade=False, cumulative=True, alpha=0.5, lw=5, 
                            label=f'injected={exp}; KS pval={np.round(pval,2)}')

            ax[i].set_title(f'{sig}')
            ax[i].set_xlim(0, 150)
            ax[i].set_xlabel('reconstructed exposure')
            ax[i].set_ylabel('cumulative density')
            ax[i].spines['top'].set_visible(False)
            ax[i].spines['right'].set_visible(False)
            ax[i].legend()
    
    if save:
        plt.savefig(f'{FIG_4}/reconstructed_vs_injected_onto_observed.all_samples.png', 
                    dpi=500, bbox_inches='tight')
    
    plt.show()    

In [ ]:
run_plot()

In [ ]:
# Compact version of the figure: close-up

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap = cm.get_cmap('Oranges')


def run_plot(save=False):

    
    
    fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(20, 3))

    foreign = ['SBS17b', 'SBS35', 'oxaliplatin', 'SBS31']
    exposures = [2, 5, 7, 10, 15, 20, 25]
    colors = list(map(cmap, [(e+4)/25 for e in exposures]))

    for i, sig in enumerate(foreign):

        v0 = np.array(reconstructed[(sig, 0)])
        g0 = sns.kdeplot(v0, ax=ax[i], bw_adjust=0.4, color='green', 
                         shade=False, cumulative=True, label='zero injected', lw=5)

        for j, exp in enumerate(exposures):

            v1 = np.array(reconstructed[(sig, exp)])
            ks, pval = scipy.stats.ks_2samp(v0, v1)
            g = sns.kdeplot(v1, ax=ax[i], bw_adjust=0.4, color=colors[j],
                            shade=False, cumulative=True, alpha=0.5, lw=5, 
                            label=f'injected={exp}; KS pval={np.round(pval,2)}')

            ax[i].set_title(f'{sig}')
            ax[i].set_xlim(-0.4, 70)
            ax[i].set_ylim(0.85,1.01)
            ax[i].set_xlabel('reconstructed exposure')
            ax[i].set_ylabel('cumulative density')
            ax[i].spines['top'].set_visible(False)
            ax[i].spines['right'].set_visible(False)
            ax[i].legend()
    
    if save:

        plt.savefig(f'{FIG_4}/reconstructed_vs_injected.all_samples.close-up.png', 
                    dpi=300, bbox_inches='tight')
        plt.savefig(f'{FIG_4}/reconstructed_vs_injected.all_samples.close-up.svg', 
                    dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
run_plot()

# 4E

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import gzip
import pickle
import json
import glob

import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

import seaborn as sns
import itertools as itt

In [ ]:
sys.path.append("../config/")
from config_plot import config_plot_params

In [ ]:
# Results parser from files
def parse_results(base_path, files):
    results_dict = {}

    for fn in glob.glob(os.path.join(base_path, files)):
        foreign = fn.split('.')[-2]
        results = pd.read_csv(fn, sep='\t')
        results.reset_index(inplace=True)
        results.rename(columns={'index': 'sample'}, inplace=True)
        results.fillna(value=0, inplace=True)
        results['sample'] = results['sample'].apply(lambda s: s.split('_')[0])
        results['log_pval'] = results['pval'].apply(lambda x: -np.log10(x))
        results_dict[foreign] = results.copy()
    
    return results_dict

# Parse mSigAct results

In [ ]:
from config import M_SIG_ACT_BASIC_FILTERS, RESULTS_BASIC_FILTERS

results_dict = parse_results(M_SIG_ACT_BASIC_FILTERS, RESULTS_BASIC_FILTERS)

# Treatment regimens

In [ ]:
from config import SIG_TREATMENT_DRUG_LVL

with gzip.open(SIG_TREATMENT_DRUG_LVL, 'rb') as f:
    treatment_regimens = pickle.load(f)

In [ ]:
carboplatin = treatment_regimens['PAN']['Carboplatin']
cisplatin = treatment_regimens['PAN']['Cisplatin']
oxaliplatin = treatment_regimens['PAN']['Oxaliplatin']
fluorouracil = treatment_regimens['PAN']['Fluorouracil']
capecitabine = treatment_regimens['PAN']['Capecitabine']

treated = {'carboplatin': carboplatin, 
           'cisplatin': cisplatin, 
           'oxaliplatin': oxaliplatin, 
           'fluorouracil': fluorouracil, 
           'capecitabine': capecitabine}

In [ ]:
# untreated

from config import UNTREATED_SAMPLES

with gzip.open(UNTREATED_SAMPLES, 'rb') as f:
    treated['untreated'] = json.load(f)

treated['platin'] = set.union(*[treated['carboplatin'], treated['cisplatin'], treated['oxaliplatin']])
treated['NMI'] = set.union(*[treated['capecitabine'], treated['fluorouracil']])

# Slice results

In [ ]:
# Recover the msigact runs with observed untreated

from config import M_SIG_ACT_OBSERVED_UNTREATED, M_SIG_ACT_CATALOGUE

results_dict_untreated = parse_results(M_SIG_ACT_OBSERVED_UNTREATED, M_SIG_ACT_CATALOGUE)

In [ ]:
def extract_sample(results):
    samples = results[foreign]['sample'].values
    samples_v = [i for i, s in enumerate(samples) if s in v]
    return results[foreign].loc[samples_v].copy()
    

treatment_slices = {}
for foreign in results_dict:
    treatment_slices[foreign] = {}
    for k, v in treated.items():
        if k == 'untreated':
            treatment_slices[foreign][k] = extract_sample(results_dict_untreated)
        else:
            treatment_slices[foreign][k] = extract_sample(results_dict)

In [ ]:
# Regimen centric analysis

from config import FIG_4

cmap = cm.get_cmap('Oranges')

def run_plot(save=False):

    for regimen in ['carboplatin', 'cisplatin', 'oxaliplatin', 'fluorouracil',
                    'capecitabine', 'untreated', 'platin', 'NMI']:

        fig, ax = plt.subplots(ncols=4, figsize=(20, 3))

        for i, foreign in enumerate(treatment_slices):

            v1 = treatment_slices[foreign][regimen][foreign].values
            sns.kdeplot(v1, ax=ax[i], bw_method=0.4, shade=False, cumulative=True, 
                        label=f'{regimen}: n={len(v1)}', color=cmap(0.9), lw=5, alpha=0.5)

            v0 = treatment_slices[foreign]['untreated'][foreign].values
            sns.kdeplot(v0, ax=ax[i], bw_method=0.4, shade=False, color='green', cumulative=True, 
                        label=f'untreated: n={len(v0)}', lw=5)

            ks, pval = scipy.stats.ks_2samp(v0, v1)

            ax[i].set_title(f'{foreign}')
            ax[i].set_ylim(0.85,)
            ax[i].set_xlabel('reconstructed exposure')
            ax[i].set_ylabel('cumulative density')
            ax[i].spines['top'].set_visible(False)
            ax[i].spines['right'].set_visible(False)

            ax[i].legend(title=f'KS pval={np.round(pval,2)}', loc=4)

        fig.suptitle(regimen)
        
        if save:
            
            plt.savefig(f'{FIG_4}/observed-treated-untreated.{regimen}.regimen-centric.close-up.svg', 
                        dpi=300, bbox_inches='tight')
            plt.savefig(f'{FIG_4}/observed-treated-untreated.{regimen}.regimen-centric.close-up.png', 
                        dpi=300, bbox_inches='tight')

    plt.show()

In [ ]:
run_plot()